In [247]:
import pandas as pd
import numpy as np

#Input file locations
lr_traces = '/Users/rufusmitchell-heggs/Desktop/H0466_ALL_corrected.csv'
timestamps = '/Users/rufusmitchell-heggs/Desktop/timestamps.csv'

def lr_data_correction(traces_or_events, timestamps):
    """Labels and corrects timings for longitudinally registered csv file of multiple sessions/stages
    
    INPUT
    -----
    traces_or_events = .csv file location for longitudinally registered events or traces
    timestamps = .csv file for timestamps of manually identified stage start and endings
    
    timestamps table format:
    ---------------------------
    |session|pre  |sam  | cho |
    ---------------------------
    | N01   |12701|21496|30611|
    ---------------------------
    
    OUPUT:
    -----
    corrected_data = A datafame containing labelled sessions and stages with corrected timings
    """
    #Read in lr_trace file location and make minor corrections
    lr_traces = pd.read_csv(lr_traces)
    lr_traces = lr_traces.drop(lr_traces.index[0])
    lr_traces = lr_traces.reset_index(drop=True)
    lr_traces = lr_traces.rename(columns={" ": "Time"})

    #Read in timestamp info
    timestamps = pd.read_csv(timestamps)
    sessions = list(timestamps['session'])
    
    #Identify start and end frames for all sessions
    all_data = list(traces_or_events['Time'].astype(float))
    session_starts = [0]
    session_ends = []
    for i in range(len(all_data)):
        if i + 1 < len(all_data):
            if abs(all_data[i+1] - all_data[i]) > 1 :
                session_starts.append(i+1)
                session_ends.append(i)
    session_ends.append(len(traces_or_events))
        
    # Save each session and each stage as a list 
    indiv_sessions = []
    for sesh, start, end in zip(sessions, session_starts, session_ends):
        
        #Isolate individual sessions
        indiv_session = traces_or_events[start:end]
        indiv_session = indiv_session.reset_index(drop=True)

        #isolate indiviudal stages within a session
        pre, sam, cho = np.array(timestamps[timestamps['session'].str.contains(sesh)])[0][1:].astype(int)
        stages = list(('PRE',) * pre) + list(('SAM',) * (sam-pre)) + list(('CHO',) * (cho-sam))    
        
        #Correct timings and add column showing stage
        stage_timings = [np.arange(0, pre, 1)*0.05006,np.arange(0, (sam-pre), 1)*0.05006,np.arange(0, (cho-sam), 1)*0.05006]
        stage_timings = [item for sublist in stage_timings for item in sublist]
        indiv_session.insert(loc=0, column='stage', value=stages)
        indiv_session['Time'] = stage_timings
        
        # Insert column showing session
        indiv_session.insert(loc=0, column='Session', value=list((sesh,) * len(indiv_session)))
        
        indiv_sessions.append(indiv_session)
    
    #Concatenate all sessions into single table
    corrected_data = pd.concat(indiv_sessions)
    return corrected_data

lr_data_traces = lr_data_correction(lr_traces, timestamps)


/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2